In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
% matplotlib inline

In [ ]:
# plt.rc('text', usetex=True)
# plt.rc('font', family='serif')
# plt.rc('font', serif='Palatino')


In [ ]:
years = [2012, 2013, 2014]

In [ ]:
df = pd.read_csv('lucentisavastin.csv')
del df['year2']
df.head()

In [ ]:
providers = {2012: {}, 2013: {}, 2014: {}}
def process(row):
    npi = row['npi']
    hcpcs_code = row['hcpcs_code']
    year = row['year']
    if npi not in providers[year]:
        providers[year][npi] = {'J2778': 0, 'J9035': 0, 'sum': 0}
    providers[year][npi][hcpcs_code] += row['bene_unique_cnt']
    providers[year][npi]['sum'] += row['bene_unique_cnt']

In [ ]:
a = df.apply(process, axis=1)

In [ ]:
lucentis_percentages = {2012: {}, 2013: {}, 2014: {}}
def percentage(row):
    npi = row['npi']
    year = row['year']
    lucentis_percentages[year][npi] = providers[year][npi]['J2778'] * 1. / providers[year][npi]['sum']

a = df.apply(percentage, axis=1)
plt.clf()
for year in years:
    plt.plot(sorted(lucentis_percentages[year].values()), label=str(year))
    plt.axvline(x=len(lucentis_percentages[year]), color='C{}'.format(year - 2012))
plt.legend(fancybox=True, shadow=True)
plt.xlabel('Ophthalmologists treating wet AMD with Lucentis and/or Avastin')
plt.ylabel('Lucentis patient share')
plt.savefig('lucentispatientshare.pdf')

In [ ]:
data = []
lucentis_prop = {}
plt.clf()
for year in years:
    year_df = pd.DataFrame.from_dict(providers[year], orient='index')
    year_df = year_df[year_df['sum'] >= 200]
    year_df['lucentis_prop'] = year_df['J2778'] * 1. / year_df['sum']
    del year_df['sum']
    year_df['J2778'] = year_df['J2778'] > 0
    year_df['J9035'] = year_df['J9035'] > 0
#     print year
#     print year_df.groupby(['J9035', 'J2778']).size()
    data.append(list(year_df.groupby(['J9035', 'J2778']).size()))
    lucentis_prop[year] = year_df['lucentis_prop']
    plt.plot(sorted(list(lucentis_prop[year])), label=str(year))
    plt.axvline(x=len(lucentis_prop[year]) + int(year == 2013), color='C{}'.format(year - 2012))

final_counts = pd.DataFrame(data, index=years, columns=['Lucentis (J2778) only', 
                                                        'Avastin (J9035) only', 
                                                        'Both'])

plt.legend(fancybox=True, shadow=True)
plt.xlabel('Ophthalmologists with more than 200 patients treated with Lucentis and/or Avastin')
plt.ylabel('Lucentis patient share')
plt.savefig('lucentispatientshare200.pdf')

In [ ]:
len(lucentis_prop[2012]), len(lucentis_prop[2013])

In [ ]:
final_props = final_counts.div(final_counts.sum(axis=1), axis=0)

In [ ]:
x = range(3)

In [ ]:
plt.clf()
plt.bar(x, 
        final_counts['Both'], 
        color='#56a0d2', 
        label='Mix')
plt.bar(x, 
        final_counts['Avastin (J9035) only'], 
        color='#ffc000', 
        label='Avastin (J9035) only',
        bottom=final_counts['Both'])
plt.bar(x, 
        final_counts['Lucentis (J2778) only'], 
        color='#99cc00', 
        label='Lucentis (J2778) only',
        bottom=final_counts['Both'] + final_counts['Avastin (J9035) only'])

plt.xticks(x, years)
plt.ylabel('Number of patients')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
          fancybox=True, shadow=True, ncol=3)
plt.title('Changes in Avastin and Lucentis Use, 2012-2014')
plt.savefig('lucentisavastinplot.pdf', bbox_inches='tight', pad_inches=0.2)

In [ ]:
with open('lucentisavastindata.tex', 'wb') as f:
    f.write((100 * final_props).to_latex(float_format='%.1f', column_format='cccc'))